In [8]:
import torch

torch.cuda.device_count()

2

In [9]:
torch.cuda.is_available()

True

In [10]:
torch.cuda.set_device(0)

In [11]:
torch.cuda.current_device()

0

In [12]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [13]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [14]:
import subprocess as sp
import os

def get_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    return memory_free_values

get_gpu_memory()

[9797, 24251]

In [15]:
import os
print(os.getcwd())

/home/jupyter-matias/Alan/newVenv/UQ-VAE/InvariantAlan/gh_uqvae


In [16]:
from dataclasses import dataclass
import numpy as np
import scipy as sp
from trainuqvae import train_uqvae
from uqvae import NNmodel
import torch
from tqdm import tqdm
from utils.quality import FoM
#import utils.genmatrices
#from utils.jwavematrix import build_sym_matrix
from utils.genmatrices import genAj
from utils.OAT import createForwMatdotdet

##############################################################################
@dataclass
class TrainingConfig:
    
    train = True
    pretrain = False
    ptepochs = 5
    continuetrain = False
    
    precalcA = True
    usesavedAj = False
    
    predict = False
    numtest = 500
    
    # UQ-VAE parameters
    #image_size = 128  # the image size and shape
    image_size = 128  # the image size and shape
    
    lamba = 0.5 # sacling parameters
    l = 0.5e-3 # [m] characteristic length
    s0 = 0.25 # prior standard deviation
    eta0 = 0.5 # prior expected value
    etae = 0 # noise mean value
    semin = 1e-3 # min value of the noise standard deviation
    semax = 5e-3 # max value of the noise standard deviation
    
    # Training parameters    
    train_batch_size = 20
    num_epochs = 10
    learning_rate = 1e-6
    val_percent = 0.2
    valid_loss_min = 100
    
    # OAT setup parameters
    Ns = 36         # number of detectors
    Nt = 1024       # number of time samples
    dx = 115e-6     # pixel size  in the x direction [m] 
    
    #nx = 128        # number of pixels in the x direction for a 2-D image region
    nx = 128
    Rs = 44e-3     # radius of the circunference where the detectors are placed [m]
    arco = 360      # arc of the circunferencewhere the detectors are placed
    vs = 1490       # speed of sound [m/s]
    to = 21.5e-6    # initial time [s].
    T = 25e-6       # durantion of the time window [s].
    tf = to + T     # final time [s] 
    LBW = True      # Apply detector impulse reponse?
    
    # 
    cache_dir = './data/' 
    traindate = '24mar24'
    
    datadate = '24mar24F'
    
    plotresults = False 
    
    ckp_last='uqvae' + traindate + '.pth' # name of the file of the saved weights of the trained net
    ckp_best='uqvae_best' + traindate + '.pth'
    
    logfilename = 'TrainingLog_UQVAE' + traindate + '.log' 

###############################################################################
def gettestndata(cache_dir,datadate):
    
    print('Obtaining data for testing...')
    
    Xt = np.load(cache_dir + 'Xdast' + datadate + '.npy')
    Yt = np.load(cache_dir + 'Yt' + datadate + '.npy') 
    #Zt = np.load(cache_dir + 'ESt' + datadate + '.npy') 
    
    Xt = Xt.astype(np.float32)
    Yt = Yt.astype(np.float32)
    #Zt = Zt.astype(np.float32)

    print('done')
    
    return Xt,Yt#,Zt
  
##############################################################################
def predict(config):
    
    numtest = config.numtest
    device = 'cpu'
    net = NNmodel().to(device=device)
    Xt,Yt = gettestndata(config.cache_dir,config.datadate)
    x = Xt[0:numtest,:,:]
    y = Yt[0:numtest,:,:]
    #z = Zt[0:numtest,:,:]
    #
    inp = torch.as_tensor(x)
    inp = inp.to(device=device)
    inp = inp.type(torch.float32)
    checkpoint = torch.load(config.ckp_best,map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])
    m, logs2 = net.predict(inp)
    
    return x,y,m,logs2

##############################################################################
def quality(test,recons):
      
    numtest,H,W = test.shape
    SSIM = np.zeros((numtest,))
    PC = np.zeros((numtest,))
    RMSE = np.zeros((numtest,))
    PSNR = np.zeros((numtest,))
    
    for i in tqdm(range(numtest)):
        SSIM[i],PC[i],RMSE[i],PSNR[i] = FoM(test[i,:,:],recons[i,:,:])
        if PC[i] == 'nan':
            PC[i] == 0
    
    print('Results:')    
    print('SSIM: ',np.mean(SSIM),np.std(SSIM))
    print('PC: ', np.mean([~np.isnan(PC)]),np.std([~np.isnan(PC)]))
    print('RMSE: ', np.mean(RMSE),np.std(RMSE))
    print('PSNR: ', np.mean(PSNR),np.std(PSNR))
        
    return SSIM, PC, RMSE, PSNR

##############################################################################
def genAo(config):
    return createForwMatdotdet( config.Ns , config.Nt , config.dx , config.nx , config.Rs , config.arco , config.vs , config.to , config.tf )


In [17]:

##############################################################################
if __name__ == '__main__':
    
    # Training hyperparameters
    config = TrainingConfig()
    
    if config.precalcA:
        a = 0
        #Aj = genAj(config)
        #Aj = genAo(config)
        #Aj = sp.sparse.csc_matrix(Aj,dtype='float32')
        #sp.sparse.save_npz(config.cache_dir+'Aj'+config.traindate+'.npz',Aj)
        
    if config.train:
        EV,TLV,VLV = train_uqvae(config)
        np.savez('lossUQVAE'+config.traindate+'.npz',E=EV,T=TLV,V=VLV)
    
    if config.predict:
        x,y,pm,plogs2 = predict(config)
        SSIM, PC, RMSE, PSNR = quality(y,pm)
        np.savez('qualityUQVAE'+config.traindate+'.npz',SSIM=SSIM,PC=PC,RMSE=RMSE,PSNR=PSNR)

Device to be used: cuda
The number of parameters of the network to be trained is: 196269954
Loading data...
Obtaining data for training...
done
Creating Prior Distribution Matrix...
Gp0:  torch.Size([16384, 16384])
done
Creating Le...
done
Creating Forward Model Matrix...
Creating SIR Matrix...
with shot noise effect reduction...
with angle sensitivity...


100%|██████████| 36864/36864 [00:32<00:00, 1136.46it/s]


Creating PA Matrix...
Applying Time Derivative Operator...
Applying detector impulse response...
Normalization...
done


Epoch 1/10:   0%|          | 0/48 [00:00<?, ?sino/s]/opt/tljh/user/lib/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/tljh/user/lib/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 1/10:   0%|          | 0/48 [00:01<?, ?sino/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 80.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 55.31 MiB is free. Process 285209 has 14.10 GiB memory in use. Including non-PyTorch memory, this process has 9.51 GiB memory in use. Of the allocated memory 7.81 GiB is allocated by PyTorch, and 131.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [25]:
!pip install jax

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pytorch-lightning 1.5.6 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [17]:
Aj.shape

(36864, 16384)